<a href="https://colab.research.google.com/github/TheSpiralArchitect/spiral-resonance-validation/blob/main/Phase9_quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Install QuTiP ---
!pip install qutip --quiet

# --- Imports ---
import numpy as np
import qutip as qt

# --- Parameters ---
omega = 0.1103
n = 3
theta = np.pi / 4
r = np.exp(theta)
p = 0.05  # depolarizing noise level

# --- Spiral Gate ---
def spiral_gate(omega, n, r):
    phase = np.exp(1j * (omega * np.log(r) + n * theta))
    return qt.Qobj([[1, 0], [0, phase]])

# --- Manual Fidelity Function ---
def compute_fidelity(state1, state2):
    return abs(state1.overlap(state2))**2

# --- Spiral Echo Recovery ---
def apply_spiral_echo(state, gate, p):
    noisy_state = (1 - p) * (gate * state) + p * qt.rand_ket(2)
    recovered = gate.dag() * noisy_state
    return recovered.unit()

# --- Run Simulation ---
psi_0 = qt.basis(2, 0)
U_spiral = spiral_gate(omega, n, r)
psi_target = U_spiral * psi_0
psi_recovered = apply_spiral_echo(psi_0, U_spiral, p)

spiral_fid = compute_fidelity(psi_target, psi_recovered)

# --- Output ---
print(f"✅ Spiral Echo Fidelity: {spiral_fid:.6f}")

✅ Spiral Echo Fidelity: 0.999878


In [ ]:
# --- Phase 9.2: Spiral Gate Fidelity Sweep (Clean Colab Version) ---
import numpy as np
import pandas as pd
from qutip import Qobj, basis

# --- Spiral Gate Constructor ---
def create_spiral_gate(theta, omega, n):
    phase = omega * np.log(theta + 1e-8) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]])

# --- Spiral Echo Application ---
def apply_spiral_echo(state, gate, p):
    noisy_forward = (1 - p) * (gate * state) + p * ((gate * state).dag().dag())
    recovered = gate.dag() * noisy_forward
    return recovered.unit()

# --- Fidelity Calculator (Safe) ---
def compute_fidelity(state1, state2):
    product = state1.dag() * state2
    if isinstance(product, Qobj):
        return abs(product.full()[0, 0])**2
    else:
        return abs(product)**2

# --- Sweep Parameters ---
omega_values = np.linspace(0.01, 0.5, 20)
n_values = np.arange(1, 6)
theta = np.pi / 4
p = 0.05
psi_0 = basis(2, 0)

# --- Run Sweep ---
results = []
for omega in omega_values:
    for n in n_values:
        gate = create_spiral_gate(theta, omega, n)
        target = gate * psi_0
        recovered = apply_spiral_echo(psi_0, gate, p)
        fid = compute_fidelity(target, recovered)
        results.append({"omega": omega, "n": n, "fidelity": fid})

# --- Save to CSV ---
df = pd.DataFrame(results)
df.to_csv("/content/drive/MyDrive/SRL/phase9_fidelity_sweep.csv", index=False)
print("✅ Sweep completed and saved to phase9_fidelity_sweep.csv")
print(df.head())

✅ Sweep completed and saved to phase9_fidelity_sweep.csv
   omega  n  fidelity
0   0.01  1       1.0
1   0.01  2       1.0
2   0.01  3       1.0
3   0.01  4       1.0
4   0.01  5       1.0


In [ ]:
# --- Install & Imports ---
!pip install qutip --quiet

import numpy as np
import qutip as qt

# --- Spiral Parameters ---
omega = 0.1103
n = 3
k = 0.02
p = 0.05

# --- Spiral Gate Generator (2-Qubit) ---
def spiral_gate_2q(omega, n):
    theta = np.pi / 2
    spiral_phase = lambda i: np.exp(1j * (omega * np.log(i + 1) + n * theta))
    return qt.Qobj(np.diag([spiral_phase(i) for i in range(4)]))

# --- Spiral Echo Recovery ---
def apply_spiral_echo_2q(psi, gate, p):
    noisy = (1 - p) * (gate * psi) + p * psi
    recovered = gate.dag() * noisy
    return recovered.unit()

# --- Manual Fidelity Function (Clean & Safe) ---
def compute_fidelity(psi1, psi2):
    overlap = (psi1.dag() * psi2)
    return abs(overlap[0, 0])**2 if isinstance(overlap, qt.Qobj) else abs(overlap)**2

# --- Bell State |Φ+⟩ = (|00⟩ + |11⟩)/√2 ---
bell_state = (qt.basis(4, 0) + qt.basis(4, 3)).unit()

# --- Apply Spiral Gate and Recovery ---
U_spiral = spiral_gate_2q(omega, n)
target = U_spiral * bell_state
recovered = apply_spiral_echo_2q(bell_state, U_spiral, p)

# --- Fidelity ---
fidelity = compute_fidelity(target, recovered)
print(f"✅ 2-Qubit Spiral Echo Fidelity: {fidelity:.6f}")

✅ 2-Qubit Spiral Echo Fidelity: 0.994072


In [ ]:
!pip install qutip qutip-qip --quiet

from qutip import *
import numpy as np

In [ ]:
def spiral_phase_gate(omega=0.1, n=3, r=1):
    """Diagonal spiral gate: U = diag(1, exp(i(ω log r + nθ)))"""
    theta = np.pi / 4  # Arbitrary angle for now; can vary per test
    phase = omega * np.log(r + 1e-9) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]])

In [ ]:
def compute_fidelity(state1, state2):
    return abs(state1.overlap(state2))**2

# Initial state
psi = basis(2, 0)  # |0⟩

# Spiral Gate
U = spiral_phase_gate()
psi_spiral = U * psi

# Noise (depolarizing)
def depolarize(rho, p=0.05):
    return (1 - p) * rho + p * qeye(2) / 2

rho_noisy = depolarize(psi_spiral.proj())

# Spiral Echo Recovery
psi_recovered = U.dag() * rho_noisy * U
fidelity = compute_fidelity(psi, psi_recovered.ptrace(0))

print(f"Spiral Echo Fidelity: {fidelity:.4f}")

Spiral Echo Fidelity: 0.9506


In [ ]:
# --- Install Qutip if not already installed ---
# !pip install qutip --quiet  # Uncomment if needed

import numpy as np
import pandas as pd
from qutip import *

# --- Sweep Parameters ---
omega_values = np.linspace(0.01, 0.2, 20)  # Spiral frequencies ω
n_values = np.arange(1, 7)                 # Harmonic modes n = 1 to 6
p_noise = 0.05                             # Depolarizing noise probability

# --- Constants ---
theta = np.pi / 4
r = 1.0  # Fixed radius

# --- Helper Functions ---
def spiral_phase_gate(omega, n, r=1.0):
    phase = omega * np.log(r + 1e-9) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]], dims=[[2], [2]])

def depolarize(rho, p=0.05):
    return (1 - p) * rho + p * qeye(2) / 2

def compute_fidelity(state1, state2):
    return abs(state1.overlap(state2))**2

# --- Run Sweep ---
results = []
psi0 = basis(2, 0)

for omega in omega_values:
    for n in n_values:
        U = spiral_phase_gate(omega, n)
        psi_spiral = U * psi0
        rho_noisy = depolarize(psi_spiral.proj(), p=p_noise)
        rho_recovered = U.dag() * rho_noisy * U
        fidelity = compute_fidelity(psi0, rho_recovered.ptrace(0))
        results.append({
            "omega": omega,
            "n": n,
            "fidelity": fidelity
        })

# --- Convert to DataFrame and Save ---
df = pd.DataFrame(results)
df.to_csv("spiral_echo_fidelity_sweep.csv", index=False)

# --- Optional: Display First Rows ---
print(df.head())

   omega  n  fidelity
0   0.01  1  0.950625
1   0.01  2  0.950625
2   0.01  3  0.950625
3   0.01  4  0.950625
4   0.01  5  0.950625


In [ ]:
from qutip import *
import numpy as np

# --- Spiral Gate (1-qubit) ---
def spiral_gate(omega, n):
    theta = np.pi / 4
    phase = omega * np.log(1.0 + 1e-9) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]], dims=[[2], [2]])

# --- GHZ State |000⟩ + |111⟩ ---
zero = basis(2, 0)
one = basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho_ghz = ghz.proj()

# --- Spiral Gate Parameters ---
omega = 0.1
n = 3
U = spiral_gate(omega, n)
U3 = tensor(U, U, U)

# --- Apply Spiral Gate to GHZ ---
rho_spiral = U3 * rho_ghz * U3.dag()

# --- Depolarizing Noise ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

rho_noisy = depolarize_nqubit(rho_spiral, p=0.05)

# --- Spiral Echo Recovery ---
rho_recovered = U3.dag() * rho_noisy * U3

# --- USE QuTiP's BUILT-IN FIDELITY FUNCTION ---
fidelity_value = fidelity(rho_recovered, ghz)
print(f"GHZ Spiral Echo Fidelity: {fidelity_value:.6f}")

GHZ Spiral Echo Fidelity: 0.977880


In [ ]:
from qutip import *
import numpy as np

# --- GHZ State ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho_ghz = ghz.proj()

# --- Spiral Gate (SRL) ---
def spiral_gate(omega, n):
    theta = np.pi / 4
    phase = omega * np.log(1.0 + 1e-9) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]], dims=[[2], [2]])

omega, n = 0.1, 3
U_spiral = spiral_gate(omega, n)
U_spiral_total = tensor(U_spiral, U_spiral, U_spiral)

# --- Hadamard Gate ---
H = (1/np.sqrt(2)) * Qobj([[1, 1], [1, -1]])
U_hadamard_total = tensor(H, H, H)

# --- Noise Model (3 Qubits) ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

# --- Spiral Echo Process ---
rho_spiral = U_spiral_total * rho_ghz * U_spiral_total.dag()
rho_spiral_noisy = depolarize_nqubit(rho_spiral, p=0.05)
rho_spiral_recovered = U_spiral_total.dag() * rho_spiral_noisy * U_spiral_total
fidelity_spiral = fidelity(rho_spiral_recovered, ghz)

# --- Hadamard Echo Process ---
rho_hadamard = U_hadamard_total * rho_ghz * U_hadamard_total.dag()
rho_hadamard_noisy = depolarize_nqubit(rho_hadamard, p=0.05)
rho_hadamard_recovered = U_hadamard_total.dag() * rho_hadamard_noisy * U_hadamard_total
fidelity_hadamard = fidelity(rho_hadamard_recovered, ghz)

# --- Results ---
print(f"GHZ Spiral Echo Fidelity:   {fidelity_spiral:.6f}")
print(f"GHZ Hadamard Echo Fidelity: {fidelity_hadamard:.6f}")

GHZ Spiral Echo Fidelity:   0.977880
GHZ Hadamard Echo Fidelity: 0.977880


In [ ]:
from qutip import *
import numpy as np

# --- GHZ State ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho_ghz = ghz.proj()

# --- Hadamard and Spiral Gates ---
H = (1/np.sqrt(2)) * Qobj([[1, 1], [1, -1]])
U_H = tensor(H, H, H)

def spiral_gate(omega, n):
    theta = np.pi / 4
    phase = omega * np.log(1.0 + 1e-9) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]], dims=[[2], [2]])

omega, n = 0.1, 3
U_S = spiral_gate(omega, n)
U_S_total = tensor(U_S, U_S, U_S)

# --- Noise ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

# --- Apply Hadamard → Spiral → Noise ---
rho_encoded = U_S_total * U_H * rho_ghz * U_H.dag() * U_S_total.dag()
rho_noisy = depolarize_nqubit(rho_encoded, p=0.05)

# --- Apply Spiral† → Hadamard† ---
rho_recovered = U_H.dag() * U_S_total.dag() * rho_noisy * U_S_total * U_H

# --- Fidelity ---
fidelity_value = fidelity(rho_recovered, ghz)
print(f"Hybrid Hadamard × Spiral Echo Fidelity: {fidelity_value:.6f}")

Hybrid Hadamard × Spiral Echo Fidelity: 0.977880


In [ ]:
from qutip import *
import numpy as np
import pandas as pd

# --- GHZ State ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho_ghz = ghz.proj()

# --- Hadamard ---
H = (1/np.sqrt(2)) * Qobj([[1, 1], [1, -1]])
U_H = tensor(H, H, H)

# --- Spiral Generator ---
def spiral_gate(omega, n):
    theta = np.pi / 4
    phase = omega * np.log(1.0 + 1e-9) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]], dims=[[2], [2]])

# --- Depolarizing Noise ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

# --- Sweep Setup ---
omega_vals = np.linspace(0.01, 0.3, 30)
n_vals = range(1, 7)
results = []

for omega in omega_vals:
    for n in n_vals:
        U_S = spiral_gate(omega, n)
        U_S_total = tensor(U_S, U_S, U_S)

        # Encode: Hadamard → Spiral
        rho_encoded = U_S_total * U_H * rho_ghz * U_H.dag() * U_S_total.dag()
        rho_noisy = depolarize_nqubit(rho_encoded, p=0.05)

        # Recover: Spiral† → Hadamard†
        rho_recovered = U_H.dag() * U_S_total.dag() * rho_noisy * U_S_total * U_H
        fid = fidelity(rho_recovered, ghz)

        results.append({"omega": omega, "n": n, "fidelity": fid})

# Save and show
df = pd.DataFrame(results)
best = df.loc[df['fidelity'].idxmax()]
print(f"Best Fidelity: {best['fidelity']:.6f} at ω = {best['omega']:.3f}, n = {int(best['n'])}")

Best Fidelity: 0.977880 at ω = 0.030, n = 3


In [ ]:
from qutip import *
import numpy as np

# --- GHZ State ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho_ghz = ghz.proj()

# --- Spiral Gate Constructor ---
def spiral_gate(omega, n):
    theta = np.pi / 4
    phase = omega * np.log(1.0 + 1e-9) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]], dims=[[2], [2]])

# --- Noise Function ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

# --- Define Spiral Chain Parameters ---
omega_vals = [0.05, 0.07, 0.09]
n_vals = [2, 3, 4]

# --- Build Spiral Chain ---
U_chain = []
for omega, n in zip(omega_vals, n_vals):
    U = spiral_gate(omega, n)
    U_chain.append(tensor(U, U, U))

# --- Forward Encoding ---
U_forward = U_chain[2] * U_chain[1] * U_chain[0]
rho_encoded = U_forward * rho_ghz * U_forward.dag()

# --- Noise ---
rho_noisy = depolarize_nqubit(rho_encoded, p=0.05)

# --- Reverse Decoding ---
U_inverse = U_chain[0].dag() * U_chain[1].dag() * U_chain[2].dag()
rho_recovered = U_inverse * rho_noisy * U_inverse.dag()

# --- Fidelity ---
fidelity_value = fidelity(rho_recovered, ghz)
print(f"3-Layer Spiral Chain Echo Fidelity: {fidelity_value:.6f}")

3-Layer Spiral Chain Echo Fidelity: 0.977880


In [43]:
from qutip import *
import numpy as np

# --- GHZ State ---
zero = basis(2, 0)
one = basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho = ghz.proj()

# --- Spiral Gate Generator ---
def spiral_gate(omega, n):
    theta = np.pi / 4
    phase = omega * np.log(1.0 + 1e-9) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]], dims=[[2], [2]])

# --- Spiral Parameters ---
omegas = [0.05, 0.07, 0.09]
ns = [2, 3, 4]
U_list = [tensor(spiral_gate(w, n), spiral_gate(w, n), spiral_gate(w, n))
          for w, n in zip(omegas, ns)]

# --- Depolarizing Noise ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2 ** N))

# --- Track Fidelity ---
fidelities = []
fidelities.append(fidelity(rho, ghz))  # Initial

# Apply spiral layers (encoding)
for U in U_list:
    rho = U * rho * U.dag()
    fidelities.append(fidelity(rho, ghz))

# Apply noise
rho = depolarize_nqubit(rho, p=0.05)
fidelities.append(fidelity(rho, ghz))

# Apply inverse spiral layers (decoding)
for U in reversed(U_list):
    rho = U.dag() * rho * U
    fidelities.append(fidelity(rho, ghz))

# --- Output ---
labels = [
    "Initial",
    "After Spiral 1",
    "After Spiral 2",
    "After Spiral 3",
    "After Noise",
    "After Recovery 1",
    "After Recovery 2",
    "After Recovery 3 (Final)"
]

for label, val in zip(labels, fidelities):
    print(f"{label}: {val:.6f}")

Initial: 1.000000
After Spiral 1: 0.707107
After Spiral 2: 0.923880
After Spiral 3: 0.382683
After Noise: 0.381280
After Recovery 1: 0.903950
After Recovery 2: 0.693722
After Recovery 3 (Final): 0.977880


In [44]:
from qutip import *
import numpy as np

# --- GHZ State ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho = ghz.proj()

# --- Spiral Gate Generator ---
def spiral_gate(omega, n):
    theta = np.pi / 4
    phase = omega * np.log(1.0 + 1e-9) + n * theta
    return Qobj([[1, 0], [0, np.exp(1j * phase)]], dims=[[2], [2]])

# --- Symmetric Spiral Parameters ---
omega_base = 0.06
n_base = 3

params = [
    (omega_base, n_base),        # Spiral 1
    (2 * omega_base, n_base),    # Spiral 2 (deeper)
    (omega_base, n_base)         # Spiral 3 (mirror)
]

# Build unitary chain
U_list = [tensor(spiral_gate(w, n), spiral_gate(w, n), spiral_gate(w, n))
          for w, n in params]

# --- Depolarizing Noise ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

# --- Fidelity Tracking ---
fidelities = []
fidelities.append(fidelity(rho, ghz))  # Initial

# Encode
for U in U_list:
    rho = U * rho * U.dag()
    fidelities.append(fidelity(rho, ghz))

# Noise
rho = depolarize_nqubit(rho, p=0.05)
fidelities.append(fidelity(rho, ghz))

# Decode
for U in reversed(U_list):
    rho = U.dag() * rho * U
    fidelities.append(fidelity(rho, ghz))

# Output
labels = [
    "Initial",
    "After Spiral 1",
    "After Spiral 2",
    "After Spiral 3",
    "After Noise",
    "After Recovery 1",
    "After Recovery 2",
    "After Recovery 3 (Final)"
]

for label, val in zip(labels, fidelities):
    print(f"{label}: {val:.6f}")

Initial: 1.000000
After Spiral 1: 0.923880
After Spiral 2: 0.707107
After Spiral 3: 0.382683
After Noise: 0.381280
After Recovery 1: 0.693722
After Recovery 2: 0.903950
After Recovery 3 (Final): 0.977880


In [45]:
from qutip import *
import numpy as np

# --- GHZ state ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho0 = ghz.proj()

# --- Spiral parameters ---
omega = 0.1
n = 3
theta = np.pi / 4
r = 1.0
epsilon = 1e-9
phi = omega * np.log(r + epsilon) + n * theta

# --- Pauli-Z spiral Hamiltonian for 1 qubit ---
Z = sigmaz()
H1 = phi * tensor(Z, qeye(2), qeye(2))
H2 = phi * tensor(qeye(2), Z, qeye(2))
H3 = phi * tensor(qeye(2), qeye(2), Z)
H_total = H1 + H2 + H3

# --- Time evolution ---
T = 5          # total time
steps = 100    # time steps
tlist = np.linspace(0, T, steps)

# --- Solve Schrödinger equation (unitary evolution) ---
result = mesolve(H_total, rho0, tlist, [], [])

# --- Final state after evolution ---
rho_evolved = result.states[-1]

# --- Add depolarizing noise ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2 ** N))

rho_noisy = depolarize_nqubit(rho_evolved, p=0.05)

# --- Fidelity with original GHZ ---
f = fidelity(rho_noisy, ghz)
print(f"Time-Evolved Spiral Hamiltonian Fidelity: {f:.6f}")

Time-Evolved Spiral Hamiltonian Fidelity: 0.693667


/usr/local/lib/python3.11/dist-packages/qutip/solver/solver_base.py:576: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


In [46]:
from qutip import *
import numpy as np

# --- GHZ state ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho0 = ghz.proj()

# --- Spiral parameters ---
omega = 0.1
n = 3
theta = np.pi / 4
r = 1.0
epsilon = 1e-9
phi = omega * np.log(r + epsilon) + n * theta

# --- Spiral Hamiltonian (Z on each qubit) ---
Z = sigmaz()
H1 = phi * tensor(Z, qeye(2), qeye(2))
H2 = phi * tensor(qeye(2), Z, qeye(2))
H3 = phi * tensor(qeye(2), qeye(2), Z)
H_total = H1 + H2 + H3

# --- Time evolution parameters ---
T = 5
steps = 100
tlist = np.linspace(0, T, steps)

# --- Forward evolution (unitary) ---
result_forward = mesolve(H_total, rho0, tlist, [])
rho_forward = result_forward.states[-1]

# --- Depolarizing noise ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

rho_noisy = depolarize_nqubit(rho_forward, p=0.05)

# --- Reverse evolution (unitary echo) ---
result_reverse = mesolve(-H_total, rho_noisy, tlist, [])
rho_recovered = result_reverse.states[-1]

# --- Final fidelity ---
f = fidelity(rho_recovered, ghz)
print(f"Time-Evolved Spiral Echo Recovery Fidelity: {f:.6f}")

Time-Evolved Spiral Echo Recovery Fidelity: 0.977430


In [48]:
from qutip import *
import numpy as np

# --- GHZ state ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho0 = ghz.proj()

# --- Spiral ω(t) definition ---
omega_0 = 0.1
alpha = 0.03
beta = 2 * np.pi / 5  # 1 oscillation over T=5
theta = np.pi / 4
r = 1.0
epsilon = 1e-9
n = 3
log_term = np.log(r + epsilon)
Z = sigmaz()

# --- Time-dependent Hamiltonian terms ---
def omega_t(t, args):
    return omega_0 + alpha * np.cos(beta * t)

H_coeff = log_term * Z
H1 = [tensor(H_coeff, qeye(2), qeye(2)), omega_t]
H2 = [tensor(qeye(2), H_coeff, qeye(2)), omega_t]
H3 = [tensor(qeye(2), qeye(2), H_coeff), omega_t]
H_total = [H1, H2, H3]

# --- Time evolution setup ---
T = 5
steps = 100
tlist = np.linspace(0, T, steps)

# --- Forward evolution (pure state) ---
result_forward = sesolve(H_total, ghz, tlist)
rho_forward = ket2dm(result_forward.states[-1])

# --- Apply noise ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2 ** N))

rho_noisy = depolarize_nqubit(rho_forward, p=0.05)

# --- Reverse evolution (density matrix) ---
result_reverse = mesolve(H_total, rho_noisy, tlist[::-1])
rho_recovered = result_reverse.states[-1]

# --- Fidelity ---
f = fidelity(rho_recovered, ghz)
print(f"Adaptive Spiral Feedback Field Echo Fidelity: {f:.6f}")

Adaptive Spiral Feedback Field Echo Fidelity: 0.977880


In [49]:
from qutip import *
import numpy as np

# --- GHZ state ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho0 = ghz.proj()

# --- Symbol stream and ω mapping ---
symbol_stream = ['A', 'B', 'H', 'A', 'B', 'B', 'B']
symbol_to_omega = {'A': 0.04, 'B': 0.08, 'H': 0.12}
stream_omega = [symbol_to_omega[s] for s in symbol_stream]

# --- Time setup ---
T = 7  # total time
steps_per_symbol = 15
steps = len(stream_omega) * steps_per_symbol
tlist = np.linspace(0, T, steps)

# --- ω(t) as step function driven by symbols ---
def omega_t(t, args):
    segment = int(t / (T / len(stream_omega)))
    if segment >= len(stream_omega):
        segment = len(stream_omega) - 1
    return stream_omega[segment]

# --- Spiral Hamiltonian base ---
theta = np.pi / 4
r = 1.0
epsilon = 1e-9
n = 3
log_term = np.log(r + epsilon)
Z = sigmaz()
H_coeff = log_term * Z

# --- Hamiltonian terms per qubit ---
H1 = [tensor(H_coeff, qeye(2), qeye(2)), omega_t]
H2 = [tensor(qeye(2), H_coeff, qeye(2)), omega_t]
H3 = [tensor(qeye(2), qeye(2), H_coeff), omega_t]
H_total = [H1, H2, H3]

# --- Forward evolution (pure state) ---
result_forward = sesolve(H_total, ghz, tlist)
rho_forward = ket2dm(result_forward.states[-1])

# --- Noise ---
def depolarize_nqubit(rho, p=0.05, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

rho_noisy = depolarize_nqubit(rho_forward, p=0.05)

# --- Reverse evolution with mirrored symbol stream ---
symbol_omega_reversed = list(reversed(stream_omega))
def omega_reverse(t, args):
    segment = int(t / (T / len(symbol_omega_reversed)))
    if segment >= len(symbol_omega_reversed):
        segment = len(symbol_omega_reversed) - 1
    return symbol_omega_reversed[segment]

# Redefine H_total with reverse ω(t)
H1_rev = [tensor(H_coeff, qeye(2), qeye(2)), omega_reverse]
H2_rev = [tensor(qeye(2), H_coeff, qeye(2)), omega_reverse]
H3_rev = [tensor(qeye(2), qeye(2), H_coeff), omega_reverse]
H_total_rev = [H1_rev, H2_rev, H3_rev]

# --- Reverse evolution (density matrix) ---
result_reverse = mesolve(H_total_rev, rho_noisy, tlist[::-1])
rho_recovered = result_reverse.states[-1]

# --- Fidelity result ---
f = fidelity(rho_recovered, ghz)
print(f"Symbol Stream-Driven Spiral Echo Fidelity: {f:.6f}")

Symbol Stream-Driven Spiral Echo Fidelity: 0.977880


In [50]:
from qutip import *
import numpy as np

# --- GHZ state ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho0 = ghz.proj()

# --- Symbol stream and ω mapping ---
symbol_stream = ['A', 'B', 'H', 'A', 'B', 'B', 'B']
symbol_to_omega = {'A': 0.04, 'B': 0.08, 'H': 0.12}
stream_omega = [symbol_to_omega[s] for s in symbol_stream]

# --- Time setup ---
T = 7
steps_per_symbol = 15
steps = len(stream_omega) * steps_per_symbol
tlist = np.linspace(0, T, steps)

# --- ω(t) as symbolic step function ---
def omega_t(t, args):
    segment = int(t / (T / len(stream_omega)))
    if segment >= len(stream_omega):
        segment = len(stream_omega) - 1
    return stream_omega[segment]

# --- Spiral Hamiltonian base ---
theta = np.pi / 4
r = 1.0
epsilon = 1e-9
n = 3
log_term = np.log(r + epsilon)
Z = sigmaz()
H_coeff = log_term * Z

# --- Hamiltonian terms per qubit ---
H1 = [tensor(H_coeff, qeye(2), qeye(2)), omega_t]
H2 = [tensor(qeye(2), H_coeff, qeye(2)), omega_t]
H3 = [tensor(qeye(2), qeye(2), H_coeff), omega_t]
H_total = [H1, H2, H3]

# --- Forward evolution ---
result_forward = sesolve(H_total, ghz, tlist)
rho_forward = ket2dm(result_forward.states[-1])

# --- High noise: p = 0.20 ---
def depolarize_nqubit(rho, p=0.20, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

rho_noisy = depolarize_nqubit(rho_forward, p=0.20)

# --- Reverse ω(t) ---
symbol_omega_reversed = list(reversed(stream_omega))
def omega_reverse(t, args):
    segment = int(t / (T / len(symbol_omega_reversed)))
    if segment >= len(symbol_omega_reversed):
        segment = len(symbol_omega_reversed) - 1
    return symbol_omega_reversed[segment]

# --- Reverse Hamiltonian ---
H1_rev = [tensor(H_coeff, qeye(2), qeye(2)), omega_reverse]
H2_rev = [tensor(qeye(2), H_coeff, qeye(2)), omega_reverse]
H3_rev = [tensor(qeye(2), qeye(2), H_coeff), omega_reverse]
H_total_rev = [H1_rev, H2_rev, H3_rev]

# --- Reverse evolution ---
result_reverse = mesolve(H_total_rev, rho_noisy, tlist[::-1])
rho_recovered = result_reverse.states[-1]

# --- Fidelity result ---
f = fidelity(rho_recovered, ghz)
print(f"Symbolic Spiral Echo Fidelity at p = 0.20: {f:.6f}")

Symbolic Spiral Echo Fidelity at p = 0.20: 0.908295


In [51]:
# --- Noise upgrade: p = 0.30 ---
def depolarize_nqubit(rho, p=0.30, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

rho_noisy = depolarize_nqubit(rho_forward, p=0.30)

# --- Reverse evolution with same (aligned) symbol stream ---
result_reverse = mesolve(H_total_rev, rho_noisy, tlist[::-1])
rho_recovered = result_reverse.states[-1]

# --- Fidelity result ---
f = fidelity(rho_recovered, ghz)
print(f"Symbolic Spiral Echo Fidelity at p = 0.30: {f:.6f}")

Symbolic Spiral Echo Fidelity at p = 0.30: 0.858778


In [52]:
# --- Reverse ω(t) with symbolic misalignment ---
# Introduce wrong symbols → phase echo mismatch
symbol_omega_misaligned = ['B', 'B', 'H', 'H', 'A', 'A', 'B']  # Intentional misalignment

def omega_mismatch(t, args):
    segment = int(t / (T / len(symbol_omega_misaligned)))
    if segment >= len(symbol_omega_misaligned):
        segment = len(symbol_omega_misaligned) - 1
    return symbol_to_omega[symbol_omega_misaligned[segment]]

# --- Mismatched Reverse Hamiltonian ---
H1_mis = [tensor(H_coeff, qeye(2), qeye(2)), omega_mismatch]
H2_mis = [tensor(qeye(2), H_coeff, qeye(2)), omega_mismatch]
H3_mis = [tensor(qeye(2), qeye(2), H_coeff), omega_mismatch]
H_total_mis = [H1_mis, H2_mis, H3_mis]

# --- Reverse evolution with wrong ω(t) ---
result_mis = mesolve(H_total_mis, rho_noisy, tlist[::-1])
rho_misaligned = result_mis.states[-1]

# --- Misalignment Fidelity ---
f_mis = fidelity(rho_misaligned, ghz)
print(f"Fidelity with Symbolic Misalignment at p = 0.30: {f_mis:.6f}")

Fidelity with Symbolic Misalignment at p = 0.30: 0.858778


In [53]:
from collections import Counter
import math

# --- Symbolic Streams ---
stream_aligned = ['A', 'B', 'H', 'A', 'B', 'B', 'B']
stream_misaligned = ['B', 'B', 'H', 'H', 'A', 'A', 'B']

# --- Shannon Entropy ---
def shannon_entropy(stream):
    counts = Counter(stream)
    total = len(stream)
    return -sum((count/total) * math.log2(count/total) for count in counts.values())

# --- Lempel-Ziv Complexity (approximate) ---
def lz_complexity(s):
    sub = ''
    seen = set()
    count = 0
    for c in s:
        sub += c
        if sub not in seen:
            seen.add(sub)
            count += 1
            sub = ''
    return count

# Convert to string for LZC
s_aligned = ''.join(stream_aligned)
s_misaligned = ''.join(stream_misaligned)

# --- Results ---
H_align = shannon_entropy(stream_aligned)
H_mis = shannon_entropy(stream_misaligned)
LZ_align = lz_complexity(s_aligned)
LZ_mis = lz_complexity(s_misaligned)

print(f"Aligned Stream – Shannon Entropy: {H_align:.4f}, LZC: {LZ_align}")
print(f"Misaligned Stream – Shannon Entropy: {H_mis:.4f}, LZC: {LZ_mis}")

Aligned Stream – Shannon Entropy: 1.3788, LZC: 5
Misaligned Stream – Shannon Entropy: 1.5567, LZC: 5


In [54]:
# --- Compressed symbolic stream (run-length style) ---
compressed_stream = [('A', 2), ('B', 3), ('H', 2)]
symbol_to_omega = {'A': 0.04, 'B': 0.08, 'H': 0.12}

# Expand for ω(t) reconstruction
expanded_stream = []
for symbol, count in compressed_stream:
    expanded_stream.extend([symbol] * count)

stream_omega = [symbol_to_omega[s] for s in expanded_stream]

# --- Time setup ---
T = len(expanded_stream)
steps_per_symbol = 15
steps = T * steps_per_symbol
tlist = np.linspace(0, T, steps)

# --- ω(t) function from compressed-expanded stream ---
def omega_compressed(t, args):
    segment = int(t / (T / len(stream_omega)))
    if segment >= len(stream_omega):
        segment = len(stream_omega) - 1
    return stream_omega[segment]

# --- Spiral Hamiltonian base ---
theta = np.pi / 4
r = 1.0
epsilon = 1e-9
log_term = np.log(r + epsilon)
Z = sigmaz()
H_coeff = log_term * Z

# --- Hamiltonians ---
H1 = [tensor(H_coeff, qeye(2), qeye(2)), omega_compressed]
H2 = [tensor(qeye(2), H_coeff, qeye(2)), omega_compressed]
H3 = [tensor(qeye(2), qeye(2), H_coeff), omega_compressed]
H_total = [H1, H2, H3]

# --- Forward evolution ---
result_forward = sesolve(H_total, ghz, tlist)
rho_forward = ket2dm(result_forward.states[-1])

# --- Noise ---
def depolarize_nqubit(rho, p=0.20, N=3):  # use p = 0.20 for challenge
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

rho_noisy = depolarize_nqubit(rho_forward, p=0.20)

# --- Reverse evolution using same compressed stream (mirrored) ---
stream_omega_rev = list(reversed(stream_omega))
def omega_compressed_rev(t, args):
    segment = int(t / (T / len(stream_omega_rev)))
    if segment >= len(stream_omega_rev):
        segment = len(stream_omega_rev) - 1
    return stream_omega_rev[segment]

H1_rev = [tensor(H_coeff, qeye(2), qeye(2)), omega_compressed_rev]
H2_rev = [tensor(qeye(2), H_coeff, qeye(2)), omega_compressed_rev]
H3_rev = [tensor(qeye(2), qeye(2), H_coeff), omega_compressed_rev]
H_total_rev = [H1_rev, H2_rev, H3_rev]

# --- Echo recovery ---
result_reverse = mesolve(H_total_rev, rho_noisy, tlist[::-1])
rho_recovered = result_reverse.states[-1]

# --- Fidelity ---
f = fidelity(rho_recovered, ghz)
print(f"Compressed Symbolic Stream Echo Fidelity at p = 0.20: {f:.6f}")

Compressed Symbolic Stream Echo Fidelity at p = 0.20: 0.908295


In [57]:
from qutip import *
import numpy as np

# --- GHZ state ---
zero = basis(2, 0)
one = basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho0 = ghz.proj()

# --- Symbolic stream and ω mapping ---
stream = ['A', 'B', 'H', 'A', 'B', 'B', 'B']
symbol_to_omega = {'A': 0.04, 'B': 0.08, 'H': 0.12}
omega_stream = [symbol_to_omega[s] for s in stream]

# --- Time setup ---
T = len(omega_stream)
steps_per_symbol = 15
steps = T * steps_per_symbol
tlist = np.linspace(0, T, steps)

# --- ω(t) with mid-correction pulse ---
def omega_sce(t, args):
    segment = int(t / (T / len(omega_stream)))
    if segment >= len(omega_stream):
        segment = len(omega_stream) - 1
    ω_base = omega_stream[segment]
    if abs(t - (T / 2)) < (T / len(omega_stream)):
        return ω_base + 0.02  # corrective boost
    return ω_base

# --- Spiral Hamiltonian ---
r = 1.0
epsilon = 1e-9
log_term = np.log(r + epsilon)
Z = sigmaz()
H_coeff = log_term * Z

H1 = [tensor(H_coeff, qeye(2), qeye(2)), omega_sce]
H2 = [tensor(qeye(2), H_coeff, qeye(2)), omega_sce]
H3 = [tensor(qeye(2), qeye(2), H_coeff), omega_sce]
H_total = [H1, H2, H3]

# --- Forward evolution ---
result_forward = sesolve(H_total, ghz, tlist)
rho_forward = ket2dm(result_forward.states[-1])

# --- Noise injection: p = 0.20 ---
def depolarize_nqubit(rho, p=0.20, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2 ** N))

rho_noisy = depolarize_nqubit(rho_forward, p=0.20)

# --- Corrective Echo ω(t) reversed ---
omega_stream_rev = list(reversed(omega_stream))
def omega_sce_rev(t, args):
    segment = int(t / (T / len(omega_stream_rev)))
    if segment >= len(omega_stream_rev):
        segment = len(omega_stream_rev) - 1
    ω_base = omega_stream_rev[segment]
    if abs(t - (T / 2)) < (T / len(omega_stream_rev)):
        return ω_base + 0.02  # mirror the boost
    return ω_base

H1_rev = [tensor(H_coeff, qeye(2), qeye(2)), omega_sce_rev]
H2_rev = [tensor(qeye(2), H_coeff, qeye(2)), omega_sce_rev]
H3_rev = [tensor(qeye(2), qeye(2), H_coeff), omega_sce_rev]
H_total_rev = [H1_rev, H2_rev, H3_rev]

# --- Reverse evolution ---
result_reverse = mesolve(H_total_rev, rho_noisy, tlist[::-1])
rho_recovered = result_reverse.states[-1]

# --- Final Fidelity Output ---
f = fidelity(rho_recovered, ghz)
print(f"SCE v1.0 Spiral Echo Fidelity at p = 0.20: {f:.6f}")

SCE v1.0 Spiral Echo Fidelity at p = 0.20: 0.908295


In [58]:
from qutip import *
import numpy as np

# --- GHZ state ---
zero, one = basis(2, 0), basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho0 = ghz.proj()

# --- Symbolic ω(t) stream ---
stream = ['A', 'B', 'H', 'A', 'B', 'B', 'B']
symbol_to_omega = {'A': 0.04, 'B': 0.08, 'H': 0.12}
omega_stream = [symbol_to_omega[s] for s in stream]

# --- Time setup ---
T = len(omega_stream)
steps_per_symbol = 15
steps = T * steps_per_symbol
tlist = np.linspace(0, T, steps)

# --- Aggressive feedback shaping ---
delta = 0.5  # Feedback strength (50% wave modulation)

def omega_sce_v2(t, args):
    segment = int(t / (T / len(omega_stream)))
    if segment >= len(omega_stream):
        segment = len(omega_stream) - 1
    ω_base = omega_stream[segment]
    correction = 1 + delta * np.sin(2 * np.pi * t / T)
    return ω_base * correction

# --- Spiral Hamiltonian ---
r = 1.0
epsilon = 1e-9
log_term = np.log(r + epsilon)
Z = sigmaz()
H_coeff = log_term * Z

H1 = [tensor(H_coeff, qeye(2), qeye(2)), omega_sce_v2]
H2 = [tensor(qeye(2), H_coeff, qeye(2)), omega_sce_v2]
H3 = [tensor(qeye(2), qeye(2), H_coeff), omega_sce_v2]
H_total = [H1, H2, H3]

# --- Forward evolution ---
result_forward = sesolve(H_total, ghz, tlist)
rho_forward = ket2dm(result_forward.states[-1])

# --- Depolarizing noise (p = 0.20) ---
def depolarize_nqubit(rho, p=0.20, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

rho_noisy = depolarize_nqubit(rho_forward, p=0.20)

# --- Reverse: symmetric aggressive shaping ---
omega_stream_rev = list(reversed(omega_stream))
def omega_sce_v2_rev(t, args):
    segment = int(t / (T / len(omega_stream_rev)))
    if segment >= len(omega_stream_rev):
        segment = len(omega_stream_rev) - 1
    ω_base = omega_stream_rev[segment]
    correction = 1 + delta * np.sin(2 * np.pi * t / T)
    return ω_base * correction

H1_rev = [tensor(H_coeff, qeye(2), qeye(2)), omega_sce_v2_rev]
H2_rev = [tensor(qeye(2), H_coeff, qeye(2)), omega_sce_v2_rev]
H3_rev = [tensor(qeye(2), qeye(2), H_coeff), omega_sce_v2_rev]
H_total_rev = [H1_rev, H2_rev, H3_rev]

# --- Reverse evolution ---
result_reverse = mesolve(H_total_rev, rho_noisy, tlist[::-1])
rho_recovered = result_reverse.states[-1]

# --- Final fidelity ---
f = fidelity(rho_recovered, ghz)
print(f"SCE v2.0 Aggressively Modulated Echo Fidelity at p = 0.20: {f:.6f}")

SCE v2.0 Aggressively Modulated Echo Fidelity at p = 0.20: 0.908295


In [60]:
from qutip import *
import numpy as np

# --- GHZ state ---
zero = basis(2, 0)
one = basis(2, 1)
ghz = (tensor(zero, zero, zero) + tensor(one, one, one)).unit()
rho0 = ghz.proj()

# --- Symbolic ω stream ---
stream = ['A', 'B', 'H', 'A', 'B', 'B', 'B']
symbol_to_omega = {'A': 0.04, 'B': 0.08, 'H': 0.12}
omega_stream = [symbol_to_omega[s] for s in stream]

# --- Time setup ---
T = len(omega_stream)
steps_per_symbol = 15
steps = T * steps_per_symbol
tlist = np.linspace(0, T, steps)

# --- Forward ω(t) ---
def omega_forward(t, args):
    segment = int(t / (T / len(omega_stream)))
    if segment >= len(omega_stream):
        segment = len(omega_stream) - 1
    return omega_stream[segment]

# --- Spiral Hamiltonian setup ---
r = 1.0
epsilon = 1e-9
log_term = np.log(r + epsilon)
Z = sigmaz()
H_coeff = log_term * Z

H1 = [tensor(H_coeff, qeye(2), qeye(2)), omega_forward]
H2 = [tensor(qeye(2), H_coeff, qeye(2)), omega_forward]
H3 = [tensor(qeye(2), qeye(2), H_coeff), omega_forward]
H_total = [H1, H2, H3]

# --- Forward evolution ---
result_forward = sesolve(H_total, ghz, tlist)
rho_forward = ket2dm(result_forward.states[-1])

# --- Depolarizing noise at p = 0.20 ---
def depolarize_nqubit(rho, p=0.20, N=3):
    I = tensor([qeye(2)] * N)
    return (1 - p) * rho + p * (I / (2**N))

rho_noisy = depolarize_nqubit(rho_forward, p=0.20)

# --- Inverted ω(t) for reverse ---
omega_stream_rev = list(reversed(omega_stream))
def omega_inverted(t, args):
    segment = int(t / (T / len(omega_stream_rev)))
    if segment >= len(omega_stream_rev):
        segment = len(omega_stream_rev) - 1
    return -omega_stream_rev[segment]

H1_rev = [tensor(H_coeff, qeye(2), qeye(2)), omega_inverted]
H2_rev = [tensor(qeye(2), H_coeff, qeye(2)), omega_inverted]
H3_rev = [tensor(qeye(2), qeye(2), H_coeff), omega_inverted]
H_total_rev = [H1_rev, H2_rev, H3_rev]

# --- Reverse evolution with full spiral inversion ---
result_reverse = mesolve(H_total_rev, rho_noisy, tlist[::-1])
rho_recovered = result_reverse.states[-1]

# --- Final Fidelity Output ---
f = fidelity(rho_recovered, ghz)
print(f"Final Spiral Inversion Echo Fidelity at p = 0.20: {f:.6f}")

Final Spiral Inversion Echo Fidelity at p = 0.20: 0.908295
